<a href="https://colab.research.google.com/github/batizza/tensortrade/blob/master/train_evaluate_batizza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install TensorTrade

In [2]:
!python3 -m pip install git+https://github.com/tensortrade-org/tensortrade.git

  Cloning https://github.com/tensortrade-org/tensortrade.git to /tmp/pip-req-build-c7vt2dmj
  Running command git clone -q https://github.com/tensortrade-org/tensortrade.git /tmp/pip-req-build-c7vt2dmj
     |████████████████████████████████| 645kB 4.2MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 788kB 28.7MB/s 
     |████████████████████████████████| 13.2MB 341kB/s 
     |████████████████████████████████| 368kB 47.7MB/s 
  Created wheel for tensortrade: filename=tensortrade-1.0.2.dev0-cp36-none-any.whl size=133806 sha256=5ed78954c1be0c39c33df39051deb4a0424ceab789e87144da0b302cd318974d
  Stored in directory: /tmp/pip-ephem-wheel-cache-vlnl5lli/wheels/de/b8/49/12c9be21a1f2c3adf64c590864587a7a30ad2a6ab8eae2ae4a
Successfully built tensortrade
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.16 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but

In [3]:
!python3 -m pip install ta
!python3 -m pip install python-binance

  Created wheel for ta: filename=ta-0.7.0-cp36-none-any.whl size=28718 sha256=ba0e2e812d27745344018c134fa89b426590a8647c0ac7fb8058abe2f1068cfd
  Stored in directory: /root/.cache/pip/wheels/dd/88/30/de9553fb54a474eb7480b937cdbb140bdda613d29cf4da7994
Successfully built ta
     |████████████████████████████████| 3.2MB 6.7MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 184kB 35.6MB/s 
     |████████████████████████████████| 491kB 30.5MB/s 
     |████████████████████████████████| 3.1MB 42.5MB/s 
     |████████████████████████████████| 286kB 47.7MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 61kB 5.3MB/s 
     |████████████████████████████████| 245kB 43.9MB/s 


## Setup Data Fetching

In [4]:
import ta
import time
import dateparser
import pytz
import json

import pandas as pd
import tensortrade.env.default as default

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed
from tensortrade.oms.exchanges import Exchange, ExchangeOptions
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC, ETH
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.agents import DQNAgent

from binance.client import Client

from datetime import datetime

%matplotlib inline

In [5]:
def date_to_milliseconds(date_str):
    """Convert UTC date to milliseconds
    If using offset strings add "UTC" to date string e.g. "now UTC", "11 hours ago UTC"
    See dateparse docs for formats http://dateparser.readthedocs.io/en/latest/
    :param date_str: date in readable format, i.e. "January 01, 2018", "11 hours ago UTC", "now UTC"
    :type date_str: str
    """
    # get epoch value in UTC
    epoch = datetime.utcfromtimestamp(0).replace(tzinfo=pytz.utc)
    # parse our date string
    d = dateparser.parse(date_str)
    # if the date is not timezone aware apply UTC timezone
    if d.tzinfo is None or d.tzinfo.utcoffset(d) is None:
        d = d.replace(tzinfo=pytz.utc)

    # return the difference in time
    return int((d - epoch).total_seconds() * 1000.0)

In [25]:
cdd = CryptoDataDownload()

api_key = 'gfIl4qz8eTer9q3v82dI7YX2WnucmKTfL6aeHQaQFZeNBixsYgvSopxtErtbjQ8t'
api_secret = 'mZyALay7Xf38YWuOuy6jNwUy52px57fO4CVHE6GtwoCydc88Ra8tLlx6MsjPlBr8'

client = Client(api_key, api_secret)
klines = client.get_klines(symbol="BTCUSDT", interval=Client.KLINE_INTERVAL_1HOUR, limit=10000, startTime=date_to_milliseconds("1 Jan, 2017"))

data = cdd.fetch("Bitstamp", "USD", "BTC", "1h")

In [26]:
data.head()

,date,unix,open,high,low,close,volume
0,2018-05-15 06:00:00,1526364000,8733.86,8796.68,8707.28,8740.99,559.93
1,2018-05-15 07:00:00,1526367600,8740.99,8766.00,8721.11,8739.00,273.58
2,2018-05-15 08:00:00,1526371200,8739.00,8750.27,8660.53,8728.49,917.79
3,2018-05-15 09:00:00,1526374800,8728.49,8754.40,8701.35,8708.32,182.62
4,2018-05-15 10:00:00,1526378400,8708.32,8865.00,8695.11,8795.90,1260.69


In [27]:
klines.pop()

[1506564000000,
 '4178.00000000',
 '4197.97000000',
 '4148.99000000',
 '4170.00000000',
 '41.00898900',
 1506567599999,
 '170846.02920384',
 609,
 '18.63784100',
 '77638.09410973',
 '9697.27837872']

In [28]:
binance_data = pd.DataFrame(klines, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'unix', 'qoute_asset_volume', 'num_trades', 'taker_base_vol', 'taker_qoute_vol', 'ignore'])
binance_data.open_time = pd.to_datetime(binance_data.open_time, unit='ms')
#binance_data.close_time = pd.to_datetime(binance_data.close_time, unit='ms')

binance_data.open = binance_data.open.astype(float)
binance_data.close = binance_data.close.astype(float)
binance_data.high = binance_data.high.astype(float)
binance_data.low = binance_data.low.astype(float)
binance_data.volume = binance_data.volume.astype(float)

binance_data.head()

,open_time,open,high,low,close,volume,unix,qoute_asset_volume,num_trades,taker_base_vol,taker_qoute_vol,ignore
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,1502945999999,202366.13839304,171,35.16050300,150952.47794304,7887.63551305
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,1502949599999,100304.82356749,102,21.44807100,92608.27972836,8039.26240152
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,1502953199999,31282.31266989,36,4.80286100,20795.31722405,8041.76049845
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,1502956799999,19241.05829986,25,2.60229200,11291.34701534,8048.12788573
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,1502960399999,4239.50358563,28,0.81465500,3552.74681715,8018.03650010


In [29]:
binance_data.open[0]

4261.48

In [31]:
data.count()

date      24210
unix      24210
open      24210
high      24210
low       24210
close     24210
volume    24210
dtype: int64

In [32]:
binance_data.count()

open_time             999
open                  999
high                  999
low                   999
close                 999
volume                999
unix                  999
qoute_asset_volume    999
num_trades            999
taker_base_vol        999
taker_qoute_vol       999
ignore                999
dtype: int64

## Create features with the feed module

In [33]:
ta.add_all_ta_features(data, 'open', 'high', 'low', 'close', 'volume', fillna=True)

features = []
for c in data.columns[1:]:
    s = Stream.source(list(data[c]), dtype="float").rename(data[c].name)
    features += [s]

feed = DataFeed(features)
feed.compile()

/usr/local/lib/python3.6/dist-packages/ta/trend.py:768: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.6/dist-packages/ta/trend.py:772: RuntimeWarning:

invalid value encountered in double_scalars



In [34]:
for i in range(5):
    print(feed.next())

{'unix': 1526364000, 'open': 8733.86, 'high': 8796.68, 'low': 8707.28, 'close': 8740.99, 'volume': 559.93, 'volume_adi': -137.6651163311054, 'volume_obv': 559.93, 'volume_cmf': -0.24586129753916633, 'volume_fi': 0.0, 'volume_mfi': 50.0, 'volume_em': 0.0, 'volume_sma_em': 0.0, 'volume_vpt': 891.24577705426, 'volume_nvi': 1000.0, 'volume_vwap': 8748.316666666666, 'volatility_atr': 0.0, 'volatility_bbm': 8740.99, 'volatility_bbh': 8740.99, 'volatility_bbl': 8740.99, 'volatility_bbw': 0.0, 'volatility_bbp': 0.0, 'volatility_bbhi': 0.0, 'volatility_bbli': 0.0, 'volatility_kcc': 8748.316666666666, 'volatility_kch': 8837.716666666667, 'volatility_kcl': 8658.916666666666, 'volatility_kcw': 2.043821763806002, 'volatility_kcp': 0.4590231170768077, 'volatility_kchi': 0.0, 'volatility_kcli': 0.0, 'volatility_dcl': 8707.28, 'volatility_dch': 8796.68, 'volatility_dcm': 8751.98, 'volatility_dcw': 1.0227674439622931, 'volatility_dcp': 0.37706935123041685, 'volatility_ui': 0.0, 'trend_macd': 0.0, 'tren

## Setup Trading Environment

In [39]:
bitstamp = Exchange("bitstamp", service=execute_order, options=ExchangeOptions(commission=0.075))(
    Stream.source(list(data["close"]), dtype="float").rename("USD-BTC")
)

portfolio = Portfolio(USD, [
    Wallet(bitstamp, 10000 * USD),
    Wallet(bitstamp, 10 * BTC)
])


renderer_feed = DataFeed([
    Stream.source(list(data["date"])).rename("date"),
    Stream.source(list(data["open"]), dtype="float").rename("open"),
    Stream.source(list(data["high"]), dtype="float").rename("high"),
    Stream.source(list(data["low"]), dtype="float").rename("low"),
    Stream.source(list(data["close"]), dtype="float").rename("close"), 
    Stream.source(list(data["volume"]), dtype="float").rename("volume"),
    Stream.source(list(data["trend_macd"]), dtype="float").rename("trend_macd"),
    Stream.source(list(data["trend_sma_fast"]), dtype="float").rename("trend_sma_fast"),
    Stream.source(list(data["trend_sma_slow"]), dtype="float").rename("trend_sma_slow"),
    Stream.source(list(data["trend_macd_signal"]), dtype="float").rename("trend_macd_signal"),
    Stream.source(list(data["trend_ema_fast"]), dtype="float").rename("trend_ema_fast"),
    Stream.source(list(data["trend_ema_slow"]), dtype="float").rename("trend_ema_slow"),
    Stream.source(list(data["trend_vortex_ind_pos"]), dtype="float").rename("trend_vortex_ind_pos"),
    Stream.source(list(data["trend_vortex_ind_neg"]), dtype="float").rename("trend_vortex_ind_neg"),
    Stream.source(list(data["trend_trix"]), dtype="float").rename("trend_trix"),
    Stream.source(list(data["trend_mass_index"]), dtype="float").rename("trend_mass_index"),
    Stream.source(list(data["trend_cci"]), dtype="float").rename("trend_cci"),
    Stream.source(list(data["trend_ichimoku_conv"]), dtype="float").rename("trend_ichimoku_conv"),
    Stream.source(list(data["trend_ichimoku_base"]), dtype="float").rename("trend_ichimoku_base"),
    Stream.source(list(data["trend_ichimoku_a"]), dtype="float").rename("trend_ichimoku_a"),
    Stream.source(list(data["trend_ichimoku_b"]), dtype="float").rename("trend_ichimoku_b") 
])


env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    feed=feed,
    renderer_feed=renderer_feed,
    renderer=default.renderers.PlotlyTradingChart(),
    window_size=20
)

In [40]:
env.observer.feed.next()

{'external': {'close': 8740.99,
  'high': 8796.68,
  'low': 8707.28,
  'momentum_ao': 0.0,
  'momentum_kama': 8740.99,
  'momentum_ppo': 0.0,
  'momentum_ppo_hist': 0.0,
  'momentum_ppo_signal': 0.0,
  'momentum_roc': 0.0,
  'momentum_rsi': 100.0,
  'momentum_stoch': 37.70693512304168,
  'momentum_stoch_rsi': 0.0,
  'momentum_stoch_rsi_d': 0.0,
  'momentum_stoch_rsi_k': 0.0,
  'momentum_stoch_signal': 37.70693512304168,
  'momentum_tsi': 0.0,
  'momentum_uo': 0.0,
  'momentum_wr': -62.29306487695832,
  'open': 8733.86,
  'others_cr': 0.0,
  'others_dlr': 0.0,
  'others_dr': -11.152689336935605,
  'trend_adx': 0.0,
  'trend_adx_neg': 0.0,
  'trend_adx_pos': 0.0,
  'trend_aroon_down': 4.0,
  'trend_aroon_ind': 0.0,
  'trend_aroon_up': 4.0,
  'trend_cci': 0.0,
  'trend_dpo': 1097.225624948347,
  'trend_ema_fast': 8740.99,
  'trend_ema_slow': 8740.99,
  'trend_ichimoku_a': 8751.98,
  'trend_ichimoku_b': 8751.98,
  'trend_ichimoku_base': 8751.98,
  'trend_ichimoku_conv': 8751.98,
  'trend_k

## Setup and Train DQN Agent

In [ ]:
agent = DQNAgent(env)

agent.train(n_steps=1000, n_episodes=30, save_path="agents/")

FigureWidget({
    'data': [{'close': array([8740.99, 8739.  , 8728.49, ..., 6251.24, 6259.  , 6238.5 ]),
    …

FigureWidget({
    'data': [{'close': array([8740.99, 8739.  , 8728.49, 8708.32, 8795.9 , 8760.  , 8760.  , 87…

FigureWidget({
    'data': [{'close': array([8740.99, 8739.  , 8728.49, 8708.32, 8795.9 , 8760.  , 8760.  , 87…

FigureWidget({
    'data': [{'close': array([8740.99, 8739.  , 8728.49, 8708.32, 8795.9 , 8760.  , 8760.  , 87…

FigureWidget({
    'data': [{'close': array([8740.99, 8739.  , 8728.49, ..., 7911.83, 7901.01, 7887.  ]),
    …

...

FigureWidget({
    'data': [{'close': array([8740.99, 8739.  , 8728.49, ..., 6599.47, 6530.  , 6502.2 ]),
    …

FigureWidget({
    'data': [{'close': array([8740.99, 8739.  , 8728.49, ..., 6461.66, 6492.26, 6474.99]),
    …

FigureWidget({
    'data': [{'close': array([8740.99, 8739.  , 8728.49, ..., 6745.98, 6747.57, 6714.91]),
    …

FigureWidget({
    'data': [{'close': array([8740.99, 8739.  , 8728.49, ..., 6724.86, 6715.68, 6704.12]),
    …

FigureWidget({
    'data': [{'close': array([8740.99, 8739.  , 8728.49, ..., 6140.6 , 6146.81, 6143.46]),
    …